<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_6/blob/main/Session6_Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import numpy as np
import csv

!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 3.9MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 71kB 4.3MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=db3b75c2c63211f08a54398c6d967ce0b993f482189584415bb8c99031bf30ef
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [2]:
import random
import googletrans
from googletrans import Translator

def translate(sentence_translated):
  # translator = Translator(service_urls=['translate.googleapis.com'])
  translator = Translator()
  # sentence = ['']

  # available_langs = list(googletrans.LANGUAGES.keys()) 
  # trans_lang = random.choice(available_langs) 
  # print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  sentence_translated = pd.DataFrame(sentence_translated.iloc[0:200,:]).reset_index().drop(columns=['index'])

  trans_array = np.array(sentence_translated['tweets'])
  # int(len(trans_array)/100)

  for i in range(200):
    translations = translator.translate(trans_array[i],src='en', dest='czech') 
    #print(translations)
    t_text = translations.text
    print('czech: ',t_text)

    translations_en_random = translator.translate(t_text, src='czech', dest='en') 
    en_text = translations_en_random.text
    print('Eng: ',en_text)
    trans_array[i] = en_text


  sentence_translated['tweets'] = trans_array
  return sentence_translated



In [3]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [4]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class encoders(nn.Module):

  def __init__(self,vocab_size, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim)
    self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
    
  def forward(self,text,text_lengths):
    embedded = self.embedding(text)
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
    packed_output, (hidden, cell) = self.encoder(packed_embedded)
    return hidden

class decoders(nn.Module):

  def __init__(self, input_to_decoder_size, decoder_hidden_size, no_times_decoder_cell_has_to_run):
    super().__init__()
    self.decoder_single_rnn_cell = nn.LSTMCell(input_to_decoder_size,decoder_hidden_size)
    self.no_times_decoder_cell_has_to_run = no_times_decoder_cell_has_to_run
    self.decoder_hidden_size = decoder_hidden_size

  def forward(self, encoder_context_vector):
    encoder_context_vector = encoder_context_vector.squeeze()
    hx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    cx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    for i in range(self.no_times_decoder_cell_has_to_run):
      hx,cx = self.decoder_single_rnn_cell(encoder_context_vector,(hx,cx))
    return hx

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, encoder, decoder, hidden_dim, output_dim):
        
      super().__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self,src,src_len):
      enc_hidden = self.encoder(src,src_len)
      #print('Encoder: ',enc_hidden)
      dec_hidden = self.decoder(enc_hidden)
      #print('Decoder: ', dec_hidden)
      dense_outputs = self.fc(dec_hidden)
      #print('----',dense_outputs[0])
      #output = F.softmax(dense_outputs,dim=1)
      #print('---',output)
      return dense_outputs

In [6]:
tweet_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweets (1).csv")
tweet_data.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [7]:
tweet_data.shape

(1364, 2)

In [8]:
tweet_data.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(tweet_data, test_size=0.3)
# (train, valid) = tweet_data.split(split_ratio=[85, 15], random_state = random.seed(SEED))
train = train.reset_index().drop(columns=['index'])
valid = valid.reset_index().drop(columns=['index'])

In [10]:
def delete(sent_delete):
  del_array = np.array(sent_delete['tweets'])
  for i in range(len(del_array)):
    words = str(del_array[i]).split()
    word_selected = random.choice(words)
    word_deleted = random_deletion(word_selected, 0.4)
    del_word = ''
    for k in word_deleted:
      del_word+=k
    del_array[i] = str(del_array[i]).replace(word_selected,del_word)
  sent_delete['tweets'] = del_array
  return sent_delete

tweet_delete = pd.DataFrame(train.iloc[:,:]).reset_index().drop(columns=['index'])
print(tweet_delete.head())
tweet_delete = delete(tweet_delete)

                                              tweets  labels
0  Obama. #drawsomething @therealdwele showing of...       0
1  RT @Stewart7Donna: Checking out "Obama Puts He...       0
2  Love the idea that @SarahPalinUSA acts like a ...       2
3  RT @RT_America Obama's pretzel logic http://t....       0
4  RT @ohgirlphrase: American kid "You're from th...       0


In [11]:
print(tweet_delete.head(10))

                                              tweets  labels
0  Obama. #drawsomething @therealdwele showing of...       0
1  RT @Stewart7Donna: Checking out "Obama Puts He...       0
2  Love the idea that @SarahPalinUSA acts like a ...       2
3  RT @RT_America Obama's pretzel logic http://t....       0
4  RT @ohgirlphrase: American kid "'r from the UK...       0
5  Texas GOP talking about going to winner-take-a...       1
6  RT @ParadigmForce: DOJ Issues Letter Clarifyin...       0
7  RT @NMsly: @marymauldin Nothg is ever a coinci...       1
8  RT @truthout: Background on ex-CIA officer Joh...       0
9  It would take far less time to sing "100 Bottl...       0


In [12]:
tweet_swap = pd.DataFrame(train.iloc[:,:]).reset_index().drop(columns=['index']) 
print(tweet_swap.head())

                                              tweets  labels
0  Obama. #drawsomething @therealdwele showing of...       0
1  RT @Stewart7Donna: Checking out "Obama Puts He...       0
2  Love the idea that @SarahPalinUSA acts like a ...       2
3  RT @RT_America Obama's pretzel logic http://t....       0
4  RT @ohgirlphrase: American kid "You're from th...       0


In [13]:
def swap(tweet_swap):
  for i in tweet_swap.index:
    words = tweet_swap['tweets'][i].split()
    if len(words) <=1:
      continue
    swapped_array = random_swap(words)
    tweet_swap['tweets'][i] = ' '.join([str(elem) for elem in swapped_array])
  return tweet_swap

In [14]:
tweet_swap = swap(tweet_swap)
tweet_swap.head(6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,off showing Obama. #drawsomething @therealdwel...,0
1,"Heat from Checking out ""Obama Puts RT on Airin...",0
2,Obama the planned for that acts like a complet...,2
3,RT http://t.co/Shv83Z0F logic @RT_America Obam...,0
4,"RT @ohgirlphrase: kid: have ""You're tea the UK...",0
5,going year talking Could Texas to winner-take-...,1


In [15]:
print(train.shape)
print(tweet_delete.shape)
print(tweet_swap.shape)

(954, 2)
(954, 2)
(954, 2)


In [16]:
tweet_translated = train
tweet_translated = translate(tweet_translated)
print(tweet_translated.shape)

czech:  Obama. #drawsomething @therealdwele předvádí http://t.co/KJc4gJso
Eng:  Obama. #drawsomething @therealdwele demonstrates http://t.co/KJc4gJso
czech:  RT @ Stewart7Donna: Kontrola „Obama klade na zprávy Fox zprávy, aby zabránila Hannity to vysílat“ v případě ústavní nouze: http://t.co/XwY62sep
Eng:  RT @ Stewart7Donna: Check "Obama puts on Fox news messages to prevent Hannity from broadcasting it" in case of constitutional emergency: http://t.co/XwY62sep
czech:  Milujte myšlenku, že @SarahPalinUSA se chová jako naprostý klaun jako utajení diamantové loupeže plánované Obamou a McCainem, pokud je to pravda.
Eng:  Love the idea that @SarahPalinUSA acts like a complete clown like the concealment of a diamond robbery planned by Obama and McCain, if that's true.
czech:  RT @RT_America Obamova preclíková logika http://t.co/Shv83Z0F @PeterSchiff #Obama campaign
Eng:  RT @RT_America Obama's pretzel logic http://t.co/Shv83Z0F @PeterSchiff #Obama campaign
czech:  RT @ohgirlphrase: Americké

In [17]:
tweet_data_1 = pd.concat([train,tweet_swap,tweet_delete,tweet_translated]).reset_index().drop(columns=['index'])
tweet_data_1.head()

,tweets,labels
0,Obama. #drawsomething @therealdwele showing of...,0
1,"RT @Stewart7Donna: Checking out ""Obama Puts He...",0
2,Love the idea that @SarahPalinUSA acts like a ...,2
3,RT @RT_America Obama's pretzel logic http://t....,0
4,"RT @ohgirlphrase: American kid ""You're from th...",0


In [18]:
tweet_data_1.labels.value_counts()

0    2021
1     842
2     199
Name: labels, dtype: int64

In [19]:
new_tweet_delete_1 = tweet_data_1[tweet_data_1.labels==1].reset_index().drop(columns=['index'])
new_tweet_delete_2 = tweet_data_1[tweet_data_1.labels==2].reset_index().drop(columns=['index'])
new_tweet_delete = pd.concat([new_tweet_delete_1,new_tweet_delete_2]).reset_index().drop(columns=['index'])
new_tweet_delete.head()

,tweets,labels
0,Texas GOP talking about going to winner-take-a...,1
1,RT @NMsly: @marymauldin Nothing is ever a coin...,1
2,RT @Talkmaster: Oh now I get it. Obama was tal...,1
3,"LaFollette ""we will go to newly elected presid...",1
4,"RT @markknoller: Saul says Pres Obama ""will do...",1


In [20]:
new_tweet_delete = delete(new_tweet_delete)
new_tweet_delete = swap(new_tweet_delete)
new_tweet_delete.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,Texas change about GOP talking to winner-take-...,1
1,is @NMsly: @marymauldin Nothing RT http://t.co...,1
2,really @Talkmaster: Oh now shorthand get it. O...,1
3,"LaFollette ""we ask go to newly president for w...",1
4,RT @markknoller: says higher Pres Obama from d...,1


In [21]:
tweet_data = pd.concat([tweet_data_1,new_tweet_delete]).reset_index().drop(columns=['index'])
print(tweet_data.head())
print(tweet_data.shape)
print(tweet_data.labels.value_counts())

                                              tweets  labels
0  Obama. #drawsomething @therealdwele showing of...       0
1  RT @Stewart7Donna: Checking out "Obama Puts He...       0
2  Love the idea that @SarahPalinUSA acts like a ...       2
3  RT @RT_America Obama's pretzel logic http://t....       0
4  RT @ohgirlphrase: American kid "You're from th...       0
(4103, 2)
0    2021
1    1684
2     398
Name: labels, dtype: int64


In [23]:
import random
import torch, torchtext
from torchtext import data
SEED = 43
torch.manual_seed(SEED)

In [24]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [25]:
fields = [('tweet', Tweet), ('label', Label)]

In [26]:
example = [torchtext.legacy.data.Example.fromlist([tweet_data.tweets[i],tweet_data.labels[i]], fields) for i in range(tweet_data.shape[0])] 

In [27]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [28]:
train_data = twitterDataset

In [29]:
print(valid.shape)
print(valid.head())

(410, 2)
                                              tweets  labels
0  With all the truth that has been exposed about...       0
1  RT @thinkprogress: Romney, who has two Harvard...       0
2  The actual Barack Obama Campaign And its parti...       0
3  RT @JeffersonObama: Arianna Huffington blastin...       1
4  President signs a bipartisan bill to help smal...       1


In [30]:
example_1 = [torchtext.legacy.data.Example.fromlist([valid.tweets[i],valid.labels[i]], fields) for i in range(valid.shape[0])] 

In [31]:
validDataset = torchtext.legacy.data.Dataset(example_1, fields)

In [32]:
valid = validDataset

In [33]:
len(train_data), len(valid)

(4103, 410)

In [34]:
vars(twitterDataset.examples[0])

{'label': 0,
 'tweet': ['Obama',
  '.',
  '#',
  'drawsomething',
  '@therealdwele',
  'showing',
  'off',
  'http://t.co/KJc4gJso']}

In [35]:
Tweet.build_vocab(train_data)
Label.build_vocab(train_data)

In [36]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  5418
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 3630), ('.', 2835), (':', 2666), ('#', 2312), (',', 2038), ('the', 1839), ('RT', 1793), ('"', 1767), ('to', 1333), ('?', 1132)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train_data, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [49]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [55]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 1
dropout = 0.4

# Instantiate the model
enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout)
dec = decoders(num_hidden_nodes,num_hidden_nodes,10)

model = classifier(enc,dec,num_hidden_nodes,num_output_nodes).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [56]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (encoder): encoders(
    (embedding): Embedding(5418, 300)
    (encoder): LSTM(300, 100, batch_first=True, dropout=0.4)
  )
  (decoder): decoders(
    (decoder_single_rnn_cell): LSTMCell(100, 100)
  )
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,867,303 trainable parameters


In [57]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [45]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [58]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.956 | Train Acc: 53.56%
	 Val. Loss: 0.726 |  Val. Acc: 70.19% 

	Train Loss: 0.700 | Train Acc: 69.69%
	 Val. Loss: 0.645 |  Val. Acc: 74.28% 

	Train Loss: 0.400 | Train Acc: 84.04%
	 Val. Loss: 0.731 |  Val. Acc: 80.77% 

	Train Loss: 0.211 | Train Acc: 92.61%
	 Val. Loss: 0.889 |  Val. Acc: 80.05% 

	Train Loss: 0.122 | Train Acc: 97.09%
	 Val. Loss: 1.007 |  Val. Acc: 79.81% 

	Train Loss: 0.084 | Train Acc: 98.23%
	 Val. Loss: 1.025 |  Val. Acc: 80.05% 

	Train Loss: 0.045 | Train Acc: 99.13%
	 Val. Loss: 1.054 |  Val. Acc: 79.81% 

	Train Loss: 0.043 | Train Acc: 98.89%
	 Val. Loss: 1.129 |  Val. Acc: 80.29% 

	Train Loss: 0.031 | Train Acc: 99.20%
	 Val. Loss: 1.224 |  Val. Acc: 77.16% 

	Train Loss: 0.015 | Train Acc: 99.71%
	 Val. Loss: 1.437 |  Val. Acc: 72.36% 



In [59]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    #prediction = model(tensor, length_tensor)
    enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout).to(device)
    encoder_output = enc(tensor,length_tensor)
    print('Encoder: ',encoder_output)
    dec = decoders(num_hidden_nodes,num_hidden_nodes,10)
    decoder_output = dec(encoder_output)
    print('Decoder: ',decoder_output)

    
    #_, pred = torch.max(prediction, 1) 
    
    #return categories[pred.item()]

In [60]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

Encoder:  tensor([[[-2.6472e-04,  1.9274e-02, -6.4385e-02,  2.1966e-01, -1.5182e-01,
          -3.2934e-01, -1.5015e-01,  8.0995e-02, -3.2503e-01, -1.4357e-01,
          -5.1871e-02,  9.7291e-02,  2.6256e-01, -3.0746e-01,  8.1730e-03,
           1.6632e-01,  1.4011e-01,  2.9486e-01, -2.0116e-01, -5.4900e-02,
           1.5134e-01, -7.9561e-03, -2.9733e-01, -2.7189e-01, -3.0584e-01,
           8.2280e-02,  3.6174e-01,  3.0220e-03, -6.0155e-02,  2.5345e-02,
           2.1745e-01,  4.3920e-03,  2.0334e-01, -2.1180e-03, -2.4491e-01,
          -3.3793e-02, -1.9443e-01,  4.7398e-01,  5.4228e-02, -5.6908e-03,
           1.0835e-01,  2.1229e-01,  2.5416e-01,  1.6178e-02, -3.3412e-02,
          -6.8919e-02, -6.0502e-02, -1.1171e-01,  3.4431e-01, -7.3872e-02,
           1.9890e-01, -5.2137e-01, -6.9903e-01, -5.3565e-02,  1.7469e-01,
           2.5398e-01, -2.6382e-01,  6.1046e-02,  2.4883e-02, -5.9994e-02,
           5.5434e-02, -1.7310e-01,  2.4660e-01, -1.9395e-01,  1.6007e-01,
           1.25

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


IndexError: ignored